## A very simple Tensorflow Logistic Regression example.

we will build a logistic regression model of some  (fake) graduate school admissions decisions.  The data consists csv file of a GRE exam score in the range 0 to 800, a grade point average in the range 0 to 4.0 and the rank of the student's undergrad institution in the range 4 to 1 (top).   The Admission Decision in binary.  

There are six columns in the data.  Ignore column 0.  the GRE, GPA and school rank are in columns 1 through 3.  Column 4 is an admission decision that is only based on the rank of the school: only students with a rank 1 school get in and others not.    Column 5 contains a more interesting (but still silly) admission model where you get in if your GRE score is 800 or your rank is 1.   in the example below we use the former model.  It is easy for the system to learn it perfectly.

The csv file is called dat.csv and it is here https://1drv.ms/u/s!AkRG9Zk_IOUag5IX1xzlv7rP9T5FFg

In [13]:
import tensorflow as tf
import numpy as np
import csv

In [14]:
sess = tf.compat.v1.InteractiveSession()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [15]:
train_datas = []
train_labels = []
test_datas = []
test_labels = []
i = 0
scale = np.array([[0.01, 1.0, 1.0]])
with open('dat.csv', 'rt',encoding='latin1') as f:
    reader = csv.reader(f)
    for row in reader:
        if i < 300:
            train_datas.append([np.float32(x) for x in row[1:4]])
            #print np.float(row[0])
            train_labels.append([np.float32(row[4])])
        else:
            test_datas.append([np.float32(x) for x in row[1:4]])
            test_labels.append([np.float(row[4])])
        i +=1

In [16]:
test_data  = np.array(test_datas, dtype=np.float32)
test_label = np.array(test_labels, dtype=np.float32)
train_data  = np.array(train_datas, dtype=np.float32)
train_label = np.array(train_labels, dtype=np.float32)

In [17]:
test_data[0:20]

array([[640.  ,   3.3 ,   2.  ],
       [660.  ,   3.6 ,   3.  ],
       [400.  ,   3.15,   2.  ],
       [680.  ,   3.98,   2.  ],
       [220.  ,   2.83,   3.  ],
       [580.  ,   3.46,   4.  ],
       [540.  ,   3.17,   1.  ],
       [580.  ,   3.51,   2.  ],
       [540.  ,   3.13,   2.  ],
       [440.  ,   2.98,   3.  ],
       [560.  ,   4.  ,   3.  ],
       [660.  ,   3.67,   2.  ],
       [660.  ,   3.77,   3.  ],
       [520.  ,   3.65,   4.  ],
       [540.  ,   3.46,   4.  ],
       [300.  ,   2.84,   2.  ],
       [340.  ,   3.  ,   2.  ],
       [780.  ,   3.63,   4.  ],
       [480.  ,   3.71,   4.  ],
       [540.  ,   3.28,   1.  ]], dtype=float32)

In [18]:
train_label[0:20]

array([[0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.]], dtype=float32)

first define the placeholders and variables

In [19]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

x = tf.placeholder(tf.float32, shape=(None,3)) 
y = tf.placeholder(tf.float32, shape =(None,1)) 

# Set model weights
W = tf.Variable(tf.zeros([3, 1]))
b = tf.Variable(tf.zeros([1]))

In [20]:
training_epochs = 400000
batch_size = 100
display_step = 1000

In [21]:
pred = tf.sigmoid(tf.matmul(x, W) + b) # Softmax
cost = tf.sqrt(tf.reduce_sum((y - pred)**2/batch_size))

opt = tf.train.AdamOptimizer()
optimizer = opt.minimize(cost)

# Initializing the variables old version is initialize_all_variable.  New version
# has global_variable_initializer.

#init = tf.global_variables_initializer()
init = tf.initialize_all_variables()

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'init' type=NoOp>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2737, in run_cell
    return result  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    return False  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2902, in run_code
    return outflag  File "<ipython-input-9-cae55e6355ed>", line 11, in <module>
    init = tf.initialize_all_variables()  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/tf_should_use.py", line 249, in wrapped
    error_in_function=error_in_function)


In [22]:
def get_batch2(batch_size, x, y):
    indices = np.random.choice(299, batch_size)
    return x[indices], y[indices]


In [23]:
sess.run(init)
# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(len(train_data)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = get_batch2(batch_size, train_data, train_label)
        # Fit training using batch data
        _, c = sess.run([optimizer, cost], feed_dict={x:batch_xs,y:batch_ys})
        # Compute average loss
        avg_cost += c / total_batch
    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print ("Epoch:", '%04d' % (epoch+1), "cost=", str(avg_cost))

print ("Optimization Finished!")


Epoch: 1000 cost= 0.18152697881062824
Epoch: 2000 cost= 0.09144687155882518
Epoch: 3000 cost= 0.05307899788022041
Epoch: 4000 cost= 0.03495782365401586
Epoch: 5000 cost= 0.022571133449673653
Epoch: 6000 cost= 0.014903961991270382
Epoch: 7000 cost= 0.007473531334350506
Epoch: 8000 cost= 0.007253767767300209
Epoch: 9000 cost= 0.0034579667262732983
Epoch: 10000 cost= 0.003720525186508894
Epoch: 11000 cost= 0.0020092947330946727
Epoch: 12000 cost= 0.002035311384436985
Epoch: 13000 cost= 0.001351064866563926
Epoch: 14000 cost= 0.0010260945030798516
Epoch: 15000 cost= 0.0008902437014815707
Epoch: 16000 cost= 0.00043464912838923436
Epoch: 17000 cost= 0.00038921140367165213
Epoch: 18000 cost= 0.0003619960625655949
Epoch: 19000 cost= 0.0003599119615197802
Epoch: 20000 cost= 0.0003100393902665625
Epoch: 21000 cost= 0.00018372688646195456
Epoch: 22000 cost= 8.39459122895884e-05
Epoch: 23000 cost= 7.08555259431402e-05
Epoch: 24000 cost= 0.00010595115357621882
Epoch: 25000 cost= 3.6476314562605694e

define a function in python to compute sigmoid of an array.

In [24]:
def sigmoida(z):
    res = np.zeros(len(z))
    for i in range(len(z)):
        y = 1.0/(1.0+np.exp(-z[i]))
        if y > 0.5:
            res[i] = 1
        else:
            res[i] = 0
    return res

In [25]:
p = sigmoida(pred.eval({x:test_data}))
q = y.eval({y:test_label})

how accurate is this. In other words whaat fraction of the cases have p = q.   

In [27]:
def accurate(p,q):
    total = 0.0
    numones = 0.
    numzeros = 0.
    for i in range(len(p)):
        if p[i] == 0:
            numzeros += 1
        else:
            numones += 1
        if p[i] == q[i]:
            total +=1
    print ("accuracy = "+str(total/len(p)))
    print ('ratio of 1s ='+str(numones/len(p)))
    print ('ratio of 0s ='+str(numzeros/len(p)))
    


In [28]:
accurate(p,q)

accuracy = 1.0
ratio of 1s =0.16
ratio of 0s =0.84


In [29]:
def softmax(v):
    k = len(v)
    res = np.zeros(k)
    tot = 0.0
    for i in range(k):
        res[i] = np.exp(v[i])
        tot += res[i]
    for i in range(k):
        res[i] = res[i]/tot
    return res

In [30]:
def sigmoid(x):
    y = 1.0/(1.0+np.exp(-x))
    if y > 0.5:
        return 1
    else:
        return 0

if we wish, we may look at the trained W and b arrays.

In [31]:
w =W.eval()

In [32]:
w

array([[ 4.9985535e-03],
       [ 4.8968406e+00],
       [-4.8550087e+01]], dtype=float32)

In [33]:
lb =b.eval()

In [34]:
lb

array([49.485455], dtype=float32)

In [36]:
for i in range(20):
    print (sigmoid(np.matmul(train_data[i], w)+lb))

0
0
1
0
0
0
1
0
0
0
0
1
1
0
1
0
0
0
0
1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
